# Intro to LLMstudio
In this tutorial you'll be running the proxy and the tracker so you can automatically log the usage metrics and costs in the default database (`llmstudio_mgmt.db`).

You'll learn:
1. How to connect to any provider available (VertexAI, OpenAI, etc.)
2. Make sync and async calls both with and without streaming
3. See the save logs

First things first:
* run `pip install llmstudio[proxy, tracker]`
* update your .env file with `GOOGLE_API_KEY` or `OPENAI_API_KEY`


## Start proxy and tracking servers
This step is to set the proxy and tracking servers.
You can opt out to start such servers (see `llmstudio-core` for the lightest version)

In [1]:
from pprint import pprint
from llmstudio.server import start_servers
from llmstudio_tracker.tracker import TrackingConfig
from llmstudio_proxy.provider import ProxyConfig

# start servers
start_servers(proxy=True, tracker=True)

proxy = ProxyConfig(host="0.0.0.0", port="8001")  # default port is 50001. set the environment varible to specify which host and port; LLMSTUDIO_ENGINE_HOST, LLMSTUDIO_ENGINE_PORT
tracking = TrackingConfig(host="0.0.0.0", port="8002")  # default port is 50002. set the environment varible to specify which host and port; LLMSTUDIO_TRACKING_HOST, LLMSTUDIO_TRACKING_PORT




Proxy server already running on 0.0.0.0:8001
Tracker server already running on 0.0.0.0:8002


In [2]:
from llmstudio.providers import LLM

vertexai = LLM("vertexai",
               proxy_config=proxy,
               tracking_config=tracking,
               session_id="vertexai-1")

openai = LLM("openai",
             proxy_config=proxy,
             tracking_config=tracking,
             session_id="openai-1")


Connected to LLMStudio Proxy @ 0.0.0.0:8001
Connected to LLMStudio Proxy @ 0.0.0.0:8001


# VertexAI
### Chat (non-stream)

In [3]:
response = vertexai.chat("What's your name", model="gemini-1.5-pro-latest")
print(response.chat_output)
pprint(response.metrics)

I am a large language model, trained by Google.  I don't have a name like a person does. 😊  

Do you have a name you'd like to call me?  Perhaps you'd like me to help you think of one? 

{'cost_usd': 0.0005474,
 'input_tokens': 4,
 'inter_token_latency_s': 0.2097789900643485,
 'latency_s': 2.672753095626831,
 'output_tokens': 52,
 'time_to_first_token_s': 1.1776690483093262,
 'tokens_per_second': 2.9931683600290766,
 'total_tokens': 56}


### Chat Async (non-stream)

In [4]:
await vertexai.achat("What's your name", model="gemini-1.5-pro-latest")

ChatCompletion(id='8bd7c2e8-d347-452f-861b-51eed141b7d9', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I am a large language model, so I don't have a name like a person does.  You can call me your helpful assistant! 😊  How can I help you today? \n", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1729705450, model='gemini-1.5-pro-latest', object='chat.completion', service_tier=None, system_fingerprint=None, usage=None, chat_input="What's your name", chat_output="I am a large language model, so I don't have a name like a person does.  You can call me your helpful assistant! 😊  How can I help you today? \n", chat_output_stream='', context=[{'role': 'user', 'content': "What's your name"}], provider='vertexai', deployment='gemini-1.5-pro-latest', timestamp=1729705451.026737, parameters={}, metrics={'input_tokens': 4, 'output_tokens': 39, 'total_tokens': 43, 'cost_usd': 0.00041089999999999996

### Chat (stream)

In [5]:
response = vertexai.chat("Write a paragfraph about space", model="gemini-1.5-pro-latest", is_stream=True)
for i, chunk in enumerate(response):
    if i%2==0:
        print("\n")
    if not chunk.metrics:
        print(chunk.chat_output_stream, end="", flush=True)
    else:
        print("\n\n## Metrics:")
        pprint(chunk.metrics)




Space, a seemingly endless tapestry of darkness speckled with brilliant celestial objects, has captivated humanity

 since we first looked up at the night sky. This vast expanse, mostly empty yet teeming with unseen forces, is home to swirling galaxies, dazzling nebulae, and fiery stars burning with unimaginable energy.  From the smallest asteroid to the largest

 black hole, space is a realm of mystery and wonder, constantly challenging our understanding of the universe and our place within it. 


## Metrics:
{'cost_usd': 0.0010423,
 'input_tokens': 8,
 'inter_token_latency_s': 0.18026659223768446,
 'latency_s': 2.8587160110473633,
 'output_tokens': 99,
 'time_to_first_token_s': 1.2113127708435059,
 'tokens_per_second': 3.4980739469592312,
 'total_tokens': 107}


### Chat Async (stream)

In [6]:
i=0
async for chunk in await vertexai.achat("Write a paragfraph about space", model="gemini-1.5-pro-latest", is_stream=True):
    if i%20==2:
        print("\n")
    if not chunk.metrics:
        print(chunk.chat_output_stream, end="", flush=True)
    else:
        print("\n\n## Metrics:")
        pprint(chunk.metrics)
    i+=1


Space, an endless expanse of darkness sprinkled with the brilliant light of stars, planets,

 and galaxies, continues to fascinate and inspire us.  It represents the ultimate frontier, filled with mystery and wonder, constantly challenging our understanding of the universe and our place within it. From the fiery birth of stars to the colossal dance of galaxies, space offers a glimpse into the raw power and beauty of the cosmos, urging us to explore, discover, and push the boundaries of human knowledge. 


## Metrics:
{'cost_usd': 0.0010423,
 'input_tokens': 8,
 'inter_token_latency_s': 0.16539198702031915,
 'latency_s': 2.9681239128112793,
 'output_tokens': 99,
 'time_to_first_token_s': 1.123415231704712,
 'tokens_per_second': 4.042957892763351,
 'total_tokens': 107}


### Get session metrics and usage

In [7]:
from llmstudio_tracker.tracker import Tracker

tracker = Tracker(tracking_config=tracking)

In [16]:
logs = tracker.get_session_logs(session_id="vertexai-1")
logs.json()[-1]


{'chat_input': 'Write a paragfraph about space',
 'chat_output': 'Space, an endless expanse of darkness sprinkled with the brilliant light of stars, planets, and galaxies, continues to fascinate and inspire us.  It represents the ultimate frontier, filled with mystery and wonder, constantly challenging our understanding of the universe and our place within it. From the fiery birth of stars to the colossal dance of galaxies, space offers a glimpse into the raw power and beauty of the cosmos, urging us to explore, discover, and push the boundaries of human knowledge. \n',
 'session_id': 'vertexai-1',
 'context': [{'role': 'user', 'content': 'Write a paragfraph about space'}],
 'provider': 'vertexai',
 'model': 'gemini-1.5-pro-latest',
 'deployment': 'gemini-1.5-pro-latest',
 'parameters': {},
 'metrics': {'input_tokens': 8,
  'output_tokens': 99,
  'total_tokens': 107,
  'cost_usd': 0.0010423,
  'latency_s': 2.9681239128112793,
  'time_to_first_token_s': 1.123415231704712,
  'inter_token

# OpenAI
### Chat (non-stream)

In [9]:
response = openai.chat("What's your name", model="gpt-4o")
print(response.chat_output)
pprint(response.metrics)

I’m an AI and don’t have a personal name, but you can call me Assistant! How can I help you today?
{'cost_usd': 0.00041,
 'input_tokens': 4,
 'inter_token_latency_s': 0.014148403097082067,
 'latency_s': 0.7885832786560059,
 'output_tokens': 26,
 'time_to_first_token_s': 0.40587806701660156,
 'tokens_per_second': 35.506712807454925,
 'total_tokens': 30}


### Chat Async (non-stream)

In [10]:
await openai.achat("What's your name", model="gpt-4o")

ChatCompletion(id='a5f215bb-9ff3-4988-a963-a3338dc8b391', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I'm ChatGPT, an AI language model created by OpenAI. How can I assist you today?", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1729705458, model='gpt-4o', object='chat.completion', service_tier=None, system_fingerprint=None, usage=None, chat_input="What's your name", chat_output="I'm ChatGPT, an AI language model created by OpenAI. How can I assist you today?", chat_output_stream='', context=[{'role': 'user', 'content': "What's your name"}], provider='openai', deployment='gpt-4o-2024-08-06', timestamp=1729705458.354282, parameters={}, metrics={'input_tokens': 4, 'output_tokens': 22, 'total_tokens': 26, 'cost_usd': 0.00035, 'latency_s': 0.5982279777526855, 'time_to_first_token_s': 0.44232702255249023, 'inter_token_latency_s': 0.007402385984148298, 'tokens_per_second': 36.775277683677

### Chat (stream)

In [11]:
response = openai.chat("Write a paragfraph about space", model="gpt-4o", is_stream=True)
for i, chunk in enumerate(response):
    if i%20==0:
        print("\n")
    if not chunk.metrics:
        print(chunk.chat_output_stream, end="", flush=True)
    else:
        print("\n\n## Metrics:")
        pprint(chunk.metrics)




Space, the vast expanse beyond Earth's atmosphere, is a realm of infinite possibilities and mysteries waiting

 to be unraveled. It encompasses everything from the glowing stars and distant galaxies to the dark void of

 interstellar space. Its boundless nature challenges our understanding and fuels our curiosity, driving scientists and explorers

, both past and present, to seek answers to profound questions about the origins of the universe, the

 potential for extraterrestrial life, and the future of humanity. As technology advances, our reach into space

 extends further, offering glimpses through telescopes and exploratory missions, hinting at the wonders that lie

 beyond. Space is not only a physical frontier but also a symbol of humanity's innate desire to discover

, explore, and push the boundaries of what is known.

## Metrics:
{'cost_usd': 0.0023650000000000003,
 'input_tokens': 8,
 'inter_token_latency_s': 0.01879954024365074,
 'latency_s': 3.3779349327087402,
 'output_t

### Chat Async (stream)

In [12]:
i=0
async for chunk in await openai.achat("Write a paragfraph about space", model="gpt-4o-mini", is_stream=True):
    if i%20==0:
        print("\n")
    if not chunk.metrics:
        print(chunk.chat_output_stream, end="", flush=True)
    else:
        print("\n\n## Metrics:")
        pprint(chunk.metrics)
    i+=1




Space, a vast and mysterious expanse that surrounds our planet, is a realm of infinite wonder

 and exploration. It stretches beyond the limits of our imagination, containing countless galaxies, stars, and celestial

 bodies, each with its own unique characteristics and stories. The silence of the cosmos contrasts sharply with the

 vibrant activities of life on Earth, yet it serves as a reminder of our place in the universe.

 Scientists and astronomers tirelessly study the cosmos, seeking to unravel its secrets and understand phenomena such as black

 holes, dark matter, and the origins of the universe itself. As technology advances, the prospect of

 exploring space becomes increasingly tangible, igniting our curiosity and inspiring innovations that may one day allow humanity to

 venture beyond our home planet to explore distant worlds and possibly encounter extraterrestrial life.

## Metrics:
{'cost_usd': 9.42e-05,
 'input_tokens': 8,
 'inter_token_latency_s': 0.0159024489231598

In [13]:
from llmstudio_tracker.tracker import Tracker

tracker = Tracker(tracking_config=tracking)

In [15]:
logs = tracker.get_session_logs(session_id="openai-1")
logs.json()[-1]

{'chat_input': 'Write a paragfraph about space',
 'chat_output': 'Space, a vast and mysterious expanse that surrounds our planet, is a realm of infinite wonder and exploration. It stretches beyond the limits of our imagination, containing countless galaxies, stars, and celestial bodies, each with its own unique characteristics and stories. The silence of the cosmos contrasts sharply with the vibrant activities of life on Earth, yet it serves as a reminder of our place in the universe. Scientists and astronomers tirelessly study the cosmos, seeking to unravel its secrets and understand phenomena such as black holes, dark matter, and the origins of the universe itself. As technology advances, the prospect of exploring space becomes increasingly tangible, igniting our curiosity and inspiring innovations that may one day allow humanity to venture beyond our home planet to explore distant worlds and possibly encounter extraterrestrial life.',
 'session_id': 'openai-1',
 'context': [{'role':